## Predicting the alpha signal using microblogging data

Your goal is to build a sentiment analysis model using the tagged data. This
sentiment analysis model should then be used to generate a new stock factor
which together with the other stock factors should be used to predict the Alpha signal

In [857]:
import pandas as pd
import numpy  as np
import json
import string

In [858]:
Json_file = 'train_data'

In [859]:
import json
with open('/Users/batchusreeharsha/Downloads/PHDdataset/train_data-1573118738755.json','r') as f:
    data = json.load(f)
trainone = pd.read_csv("train_factors-1573207730757.csv",sep=',', header=0, na_values='unknown')    
with open('/Users/batchusreeharsha/Downloads/PHDdataset/test_data.json','r') as f:
    datatwo = json.load(f)
traintwo = pd.read_csv("test_factors.csv",sep=',', header=0, na_values='unknown')     

In [860]:
data

{'records': [{'stocktwit_tweet': '$AMD going up but hesitating however chart is very stable and going upward',
   'sentiment_score': 3,
   'timestamp': '2018-09-19 18:38:28+00:00',
   'ticker': '$AMD'},
  {'stocktwit_tweet': '@inforlong @MariaGascon Despite\nChina trade war $CAT held very well 👍',
   'sentiment_score': 3,
   'timestamp': '2018-10-09 03:51:06+00:00',
   'ticker': '$CAT'},
  {'stocktwit_tweet': '$AVGO WTF?',
   'sentiment_score': 2,
   'timestamp': '2018-07-12 13:35:32+00:00',
   'ticker': '$AVGO'},
  {'stocktwit_tweet': '$PH\n New Insider Filing On: \n MULLER KLAUS PETER\nTransaction Code: \n http://www.filingscanner.com/Alerts/PH.php',
   'sentiment_score': 2,
   'timestamp': '2018-07-19 03:32:50+00:00',
   'ticker': '$PH'},
  {'stocktwit_tweet': '$FB if it bounces tommorrow do the right thing and GTFO',
   'sentiment_score': 3,
   'timestamp': '2018-08-23 19:07:54+00:00',
   'ticker': '$FB'},
  {'stocktwit_tweet': '$FB as usual......RED  \nevery day.',
   'sentiment_s

In [861]:
datatwo

{'records': [{'stocktwit_tweet': '$CELG nothing to be exited about',
   'timestamp': '2018-10-25 14:26:16+00:00',
   'ticker': '$CELG'},
  {'stocktwit_tweet': '$AMD yall exhaust your buyer on first green candle,,,, byeeeeee',
   'timestamp': '2018-07-13 13:50:39+00:00',
   'ticker': '$AMD'},
  {'stocktwit_tweet': '$AMD day traders day.',
   'timestamp': '2018-09-25 19:10:54+00:00',
   'ticker': '$AMD'},
  {'stocktwit_tweet': '$CBS https://tenor.com/wLB8.gif',
   'timestamp': '2018-07-27 22:45:48+00:00',
   'ticker': '$CBS'},
  {'stocktwit_tweet': '$MU weak price action so far today. Don’t be afraid to go short, it’s gonna break support and free fall soon',
   'timestamp': '2018-07-31 14:59:06+00:00',
   'ticker': '$MU'},
  {'stocktwit_tweet': '$AMZN continues to grow, specifically in key areas like cloud computing, where its profit margins are huge, and offline retail💰',
   'timestamp': '2018-08-04 23:54:24+00:00',
   'ticker': '$AMZN'},
  {'stocktwit_tweet': '$CLF If you are not loadi

In [862]:
from pandas.io.json import json_normalize
json_data = pd.concat([pd.DataFrame(data),pd.DataFrame(list(data['records']))],axis=1).drop('records',1)
type(json_data)

pandas.core.frame.DataFrame

In [863]:
from pandas.io.json import json_normalize
json_datatwo = pd.concat([pd.DataFrame(datatwo),pd.DataFrame(list(datatwo['records']))],axis=1).drop('records',1)
type(json_datatwo)

pandas.core.frame.DataFrame

In [864]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00


In [865]:
pwd()

'/Users/batchusreeharsha/Downloads/PHDdataset'

In [866]:
type(data)

dict

In [867]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha
0,1,21/08/18,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2
1,2,11/10/18,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3
2,3,21/08/18,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1
3,4,10/07/18,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2
4,5,12/09/18,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2


In [868]:
print(trainone.shape,'\n')

(27006, 11) 



In [869]:
trainone.describe()

,Id,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha
count,27006.000000,27006.000000,27006.000000,27006.000000,27006.000000,27006.000000,27006.000000,27006.000000,27006.000000
mean,13503.500000,0.005429,-0.001005,-0.024421,-0.042420,0.041465,0.002597,0.036744,2.379582
std,7796.105021,0.941853,0.996130,0.539555,1.111309,0.976028,0.998839,1.204473,1.008681
min,1.000000,-4.114670,-4.578587,-2.609150,-6.329798,-5.480268,-4.338751,-5.347869,1.000000
25%,6752.250000,-0.596792,-0.666310,-0.300187,-0.767247,-0.518641,-0.668027,-0.688198,2.000000
50%,13503.500000,0.012104,-0.013257,-0.040786,-0.058629,0.072127,0.007417,0.081374,2.000000
75%,20254.750000,0.611574,0.669094,0.241424,0.677433,0.616538,0.674592,0.807556,3.000000
max,27006.000000,4.415048,3.822270,3.452486,4.907407,4.779911,4.075428,5.182912,4.000000


In [870]:
trainone.dtypes

Id          int64
date       object
ticker     object
SF1       float64
SF2       float64
SF3       float64
SF4       float64
SF5       float64
SF6       float64
SF7       float64
alpha       int64
dtype: object

In [871]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha
0,1,21/08/18,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2
1,2,11/10/18,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3
2,3,21/08/18,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1
3,4,10/07/18,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2
4,5,12/09/18,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2


In [872]:
trainone['day'] = trainone['date'].str.split('/', expand = True)[0]
trainone['month'] = trainone['date'].str.split('/',expand = True)[1]

In [873]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,21/08/18,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,11/10/18,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,21/08/18,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,10/07/18,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,12/09/18,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [874]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,21/08/18,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,11/10/18,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,21/08/18,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,10/07/18,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,12/09/18,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [875]:
trainone['date']=pd.to_datetime(trainone['date'].astype(str),format="%d/%m/%y")

In [876]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,2018-08-21,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,2018-10-11,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,2018-08-21,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,2018-07-10,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,2018-09-12,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [877]:
##corr = trainone.corr()
#ax = sns.heatmap(corr,annot =True, linewidth = 5, cbar = True)

## Emoji to Words

In [878]:
import emoji

json_data['tidy_Tweet']=json_data['stocktwit_tweet'].apply(lambda x: emoji.demojize(x))

In [879]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...


## Removal of Punctuations

In [880]:
#json_data.drop(["stocktwit_tweet"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

json_data["tidy_Tweet_wo_punct"] = json_data["tidy_Tweet"].apply(lambda text: remove_punctuation(text))
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...


## Removal of Stopwords

In [881]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [882]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

json_data["stocktwit_tweet_wo_stop"] = json_data["tidy_Tweet_wo_punct"].apply(lambda text: remove_stopwords(text))
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,AMD going hesitating however chart stable goin...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,inforlong MariaGascon Despite China trade war ...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,AVGO WTF
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,PH New Insider Filing On MULLER KLAUS PETER Tr...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,FB bounces tommorrow right thing GTFO


## Removal of Frequent words

In [883]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,AMD going hesitating however chart stable goin...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,inforlong MariaGascon Despite China trade war ...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,AVGO WTF
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,PH New Insider Filing On MULLER KLAUS PETER Tr...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,FB bounces tommorrow right thing GTFO


In [884]:
from collections import Counter
cnt = Counter()
for stocktwit_tweet in json_data["stocktwit_tweet_wo_stop"].values:
    for word in stocktwit_tweet.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('AMD', 176119),
 ('MU', 88248),
 ('I', 84700),
 ('AMZN', 70520),
 ('FB', 69576),
 ('AAPL', 51849),
 ('NFLX', 47030),
 ('short', 37928),
 ('today', 37598),
 ('Estimize', 34073)]

## Tokenization

In [885]:
from nltk.tokenize.toktok import ToktokTokenizer

In [886]:
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/batchusreeharsha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [887]:
token_text_test = []

for i in json_data['stocktwit_tweet_wo_stop']:
    token_text_test.append(word_tokenize(i))
    
json_data['stocktwit_tweet_wo_stop'] = token_text_test

## Lemmatization

In [888]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

json_data['stocktwit_tweet_wo_stop_Tweet'] = json_data['stocktwit_tweet_wo_stop'].apply(lambda x: [lemmatizer.lemmatize(y,pos = 'v') for y in x])

json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...","[AMD, go, hesitate, however, chart, stable, go..."
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...","[inforlong, MariaGascon, Despite, China, trade..."
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]","[AVGO, WTF]"
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...","[PH, New, Insider, Filing, On, MULLER, KLAUS, ..."
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]","[FB, bounce, tommorrow, right, thing, GTFO]"


## Removing  brackets

In [889]:
json_data['stocktwit_tweet_wo_stop_Tweet'] =json_data['stocktwit_tweet_wo_stop_Tweet'].apply(lambda x: ' '.join(x))
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO


In [890]:
json_data['date'] = json_data['timestamp'].str.split(' ',expand=True)[0]

In [891]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23


In [892]:
json_data['day'] = json_data['date'].str.split('-',expand=True)[2]

In [893]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19,19
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09,09
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12,12
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19,19
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23,23


In [894]:
json_data['month'] = json_data['date'].str.split('-',expand=True)[1]

In [895]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19,19,09
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09,09,10
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12,12,07
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19,19,07
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23,23,08


In [896]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,2018-08-21,$NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,2018-10-11,$WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,2018-08-21,$DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,2018-07-10,$ge,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,2018-09-12,$FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [897]:
json_data.shape

(1039131, 11)

In [898]:
trainone['ticker']=trainone['ticker'].str.replace('$','')
trainone['ticker']=trainone['ticker'].str.upper()

In [900]:
json_data['ticker']=json_data['ticker'].str.replace('$','')
json_data['ticker']=json_data['ticker'].str.upper()

In [902]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,2018-08-21,NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,2018-10-11,WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,2018-08-21,DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,2018-07-10,GE,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,2018-09-12,FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [903]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,3,$AMD going up but hesitating however chart is ...,AMD,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19,19,09
1,3,@inforlong @MariaGascon Despite\nChina trade w...,CAT,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09,09,10
2,2,$AVGO WTF?,AVGO,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12,12,07
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19,19,07
4,3,$FB if it bounces tommorrow do the right thing...,FB,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23,23,08


In [905]:
new_json2=json_data
new_train2=trainone

In [909]:
new_json2=new_json2.groupby(by=['day','month','ticker'])['sentiment_score'].agg(lambda x:x.value_counts().index[0]).to_frame()
train_merge_work = pd.merge(new_train2, new_json2,how='left',left_on = ['ticker','day','month'],right_on=['ticker','day','month'])

In [910]:
train_merge_work.isnull().sum()


Id                    0
date                  0
ticker                0
SF1                   0
SF2                   0
SF3                   0
SF4                   0
SF5                   0
SF6                   0
SF7                   0
alpha                 0
day                   0
month                 0
sentiment_score    2167
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [765]:
le = preprocessing.LabelEncoder()
json_data['ticker'] = le.fit_transform(json_data.ticker.values)


In [766]:
for col in ('stocktwit_tweet_wo_stop_Tweet','sentiment_score'):
    json_data[col] = json_data[col].astype('category')

In [767]:
from sklearn.model_selection import train_test_split

In [780]:
x = json_data["stocktwit_tweet_wo_stop_Tweet"]
y = json_data["sentiment_score"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30,random_state=123)

In [781]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()


In [782]:
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)

In [783]:
Test_data = json_data['stocktwit_tweet_wo_stop_Tweet']
Vectorized_test_data = cv.transform(Test_data)

In [784]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [785]:
RF_classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=2)

param_grid = {'n_estimators' : [5,7,9,11,13,15,17,19],
             'max_depth' : [2,3,4,5,6,7,8,9,10,11,12],
              "min_samples_leaf" : [5, 10, 15, 20],
              "class_weight" : ['balanced','balanced_subsample']}

In [786]:
Rf = RandomizedSearchCV(estimator=RF_classifier, param_distributions=param_grid, n_iter=10,cv=4)

In [787]:
Rf.fit(x_train,y_train)

RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=10,
                                                    n_jobs=None,

In [788]:
train_rf_pred = Rf.predict(x_train)
test_rf_pred = Rf.predict(x_test)

print(Rf.score(x_train, y_train))
print(Rf.score(x_test, y_test))


0.4749357635714492
0.4735452620773722


In [789]:
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00


## Emoji to Words for jsontwo

In [790]:
json_datatwo['tidy_Tweet']=json_datatwo['stocktwit_tweet'].apply(lambda x: emoji.demojize(x))

In [791]:
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...


## Removal of Punctuations

In [792]:
#json_data.drop(["stocktwit_tweet"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

json_datatwo["tidy_Tweet_wo_punct"] = json_datatwo["tidy_Tweet"].apply(lambda text: remove_punctuation(text))
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...


## Removal of Stopwords

In [793]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [794]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

json_datatwo["stocktwit_tweet_wo_stop"] = json_datatwo["tidy_Tweet_wo_punct"].apply(lambda text: remove_stopwords(text))
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,CELG nothing exited
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,AMD yall exhaust buyer first green candle byee...
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,AMD day traders day
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,CBS httpstenorcomwLB8gif
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,MU weak price action far today Don’t afraid go...


In [ ]:
json_data.head()

## Removal of Frequent words

In [796]:
from collections import Counter
cnt = Counter()
for stocktwit_tweet in json_datatwo["stocktwit_tweet_wo_stop"].values:
    for word in stocktwit_tweet.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('AMD', 45325),
 ('MU', 22603),
 ('I', 21614),
 ('AMZN', 17845),
 ('FB', 17615),
 ('AAPL', 13249),
 ('NFLX', 12005),
 ('today', 9584),
 ('short', 9582),
 ('Estimize', 8670)]

## Tokenization

In [797]:
from nltk.tokenize.toktok import ToktokTokenizer

In [798]:
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/batchusreeharsha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [799]:
token_text_test = []

for i in json_datatwo['stocktwit_tweet_wo_stop']:
    token_text_test.append(word_tokenize(i))
    
json_datatwo['stocktwit_tweet_wo_stop'] = token_text_test

## Lemmatization

In [800]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

json_datatwo['stocktwit_tweet_wo_stop_Tweet'] = json_datatwo['stocktwit_tweet_wo_stop'].apply(lambda x: [lemmatizer.lemmatize(y,pos = 'v') for y in x])

json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]","[CELG, nothing, exit]"
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...","[AMD, yall, exhaust, buyer, first, green, cand..."
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]","[AMD, day, traders, day]"
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]","[CBS, httpstenorcomwLB8gif]"
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...","[MU, weak, price, action, far, today, Don, ’, ..."


## Removing brackets

In [801]:
json_datatwo['stocktwit_tweet_wo_stop_Tweet'] =json_datatwo['stocktwit_tweet_wo_stop_Tweet'].apply(lambda x: ' '.join(x))
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...


In [802]:
json_datatwo['date'] = json_datatwo['timestamp'].str.split(' ',expand=True)[0]
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit,2018-10-25
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...,2018-07-13
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day,2018-09-25
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif,2018-07-27
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...,2018-07-31


In [804]:
json_datatwo['day'] = json_datatwo['date'].str.split('-',expand=True)[2]
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit,2018-10-25,25
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...,2018-07-13,13
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day,2018-09-25,25
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif,2018-07-27,27
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...,2018-07-31,31


In [805]:
json_datatwo['month'] = json_datatwo['date'].str.split('-',expand=True)[1]
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit,2018-10-25,25,10
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...,2018-07-13,13,07
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day,2018-09-25,25,09
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif,2018-07-27,27,07
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...,2018-07-31,31,07


In [806]:
json_datatwo.shape

(265022, 10)

In [807]:
json_datatwo['ticker']=json_datatwo['ticker'].str.replace('$','')
json_datatwo['ticker']=json_datatwo['ticker'].str.upper()

In [808]:
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,$CELG nothing to be exited about,CELG,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit,2018-10-25,25,10
1,$AMD yall exhaust your buyer on first green ca...,AMD,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...,2018-07-13,13,07
2,$AMD day traders day.,AMD,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day,2018-09-25,25,09
3,$CBS https://tenor.com/wLB8.gif,CBS,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif,2018-07-27,27,07
4,$MU weak price action so far today. Don’t be a...,MU,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...,2018-07-31,31,07


In [810]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,3,$AMD going up but hesitating however chart is ...,68,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19,19,09
1,3,@inforlong @MariaGascon Despite\nChina trade w...,212,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09,09,10
2,2,$AVGO WTF?,129,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12,12,07
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,984,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19,19,07
4,3,$FB if it bounces tommorrow do the right thing...,461,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23,23,08


In [812]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [813]:
le = preprocessing.LabelEncoder()
json_datatwo['ticker'] = le.fit_transform(json_datatwo.ticker.values)


In [818]:
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,$CELG nothing to be exited about,158,2018-10-25 14:26:16+00:00,$CELG nothing to be exited about,CELG nothing to be exited about,"[CELG, nothing, exited]",CELG nothing exit,2018-10-25,25,10
1,$AMD yall exhaust your buyer on first green ca...,48,2018-07-13 13:50:39+00:00,$AMD yall exhaust your buyer on first green ca...,AMD yall exhaust your buyer on first green can...,"[AMD, yall, exhaust, buyer, first, green, cand...",AMD yall exhaust buyer first green candle byee...,2018-07-13,13,07
2,$AMD day traders day.,48,2018-09-25 19:10:54+00:00,$AMD day traders day.,AMD day traders day,"[AMD, day, traders, day]",AMD day traders day,2018-09-25,25,09
3,$CBS https://tenor.com/wLB8.gif,151,2018-07-27 22:45:48+00:00,$CBS https://tenor.com/wLB8.gif,CBS httpstenorcomwLB8gif,"[CBS, httpstenorcomwLB8gif]",CBS httpstenorcomwLB8gif,2018-07-27,27,07
4,$MU weak price action so far today. Don’t be a...,561,2018-07-31 14:59:06+00:00,$MU weak price action so far today. Don’t be a...,MU weak price action so far today Don’t be afr...,"[MU, weak, price, action, far, today, Don, ’, ...",MU weak price action far today Don ’ t afraid ...,2018-07-31,31,07


In [820]:
Test_data = json_datatwo['stocktwit_tweet_wo_stop_Tweet']
Vectorized_test_data = cv.transform(Test_data)

In [982]:
json_datatwo['sentiment_score']=test_json_Predictions

In [822]:
json_data.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,tidy_Tweet,tidy_Tweet_wo_punct,stocktwit_tweet_wo_stop,stocktwit_tweet_wo_stop_Tweet,date,day,month
0,3,$AMD going up but hesitating however chart is ...,68,2018-09-19 18:38:28+00:00,$AMD going up but hesitating however chart is ...,AMD going up but hesitating however chart is v...,"[AMD, going, hesitating, however, chart, stabl...",AMD go hesitate however chart stable go upward,2018-09-19,19,09
1,3,@inforlong @MariaGascon Despite\nChina trade w...,212,2018-10-09 03:51:06+00:00,@inforlong @MariaGascon Despite\nChina trade w...,inforlong MariaGascon Despite\nChina trade war...,"[inforlong, MariaGascon, Despite, China, trade...",inforlong MariaGascon Despite China trade war ...,2018-10-09,09,10
2,2,$AVGO WTF?,129,2018-07-12 13:35:32+00:00,$AVGO WTF?,AVGO WTF,"[AVGO, WTF]",AVGO WTF,2018-07-12,12,07
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,984,2018-07-19 03:32:50+00:00,$PH\n New Insider Filing On: \n MULLER KLAUS P...,PH\n New Insider Filing On \n MULLER KLAUS PET...,"[PH, New, Insider, Filing, On, MULLER, KLAUS, ...",PH New Insider Filing On MULLER KLAUS PETER Tr...,2018-07-19,19,07
4,3,$FB if it bounces tommorrow do the right thing...,461,2018-08-23 19:07:54+00:00,$FB if it bounces tommorrow do the right thing...,FB if it bounces tommorrow do the right thing ...,"[FB, bounces, tommorrow, right, thing, GTFO]",FB bounce tommorrow right thing GTFO,2018-08-23,23,08


In [853]:
trainone.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month
0,1,2018-08-21,NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08
1,2,2018-10-11,WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10
2,3,2018-08-21,DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08
3,4,2018-07-10,GE,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07
4,5,2018-09-12,FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09


In [855]:
trainone.dtypes

Id                 int64
date      datetime64[ns]
ticker          category
SF1              float64
SF2              float64
SF3              float64
SF4              float64
SF5              float64
SF6              float64
SF7              float64
alpha              int64
day             category
month           category
dtype: object

In [856]:
json_data.dtypes

sentiment_score                  category
stocktwit_tweet                    object
ticker                           category
timestamp                          object
tidy_Tweet                         object
tidy_Tweet_wo_punct                object
stocktwit_tweet_wo_stop            object
stocktwit_tweet_wo_stop_Tweet    category
date                               object
day                              category
month                            category
dtype: object

In [913]:
train_merge_work.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,day,month,sentiment_score
0,1,2018-08-21,NTAP,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,21,08,2.0
1,2,2018-10-11,WYNN,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,11,10,2.0
2,3,2018-08-21,DRI,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,21,08,2.0
3,4,2018-07-10,GE,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,10,07,2.0
4,5,2018-09-12,FE,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,12,09,0.0


In [915]:
train_merge_work.isnull().sum()

Id                    0
date                  0
ticker                0
SF1                   0
SF2                   0
SF3                   0
SF4                   0
SF5                   0
SF6                   0
SF7                   0
alpha                 0
day                   0
month                 0
sentiment_score    2167
dtype: int64

In [918]:
train_merge_work.head()

,SF1,SF2,SF3,SF4,SF5,SF6,SF7,alpha,sentiment_score
0,-0.628652,0.988891,-0.055714,0.774379,0.551089,-1.329229,-0.995539,2,2.0
1,1.315786,1.438754,0.187327,0.608933,-1.153030,1.859441,0.730995,3,2.0
2,-1.141388,-1.455016,0.332755,0.674502,0.111326,-0.478597,-1.488157,1,2.0
3,-0.054839,-1.454149,-0.162267,-0.681870,0.307869,-0.529987,0.404172,2,2.0
4,-0.686366,0.838865,0.073830,0.679024,0.329463,1.262782,-1.024042,2,0.0


In [916]:
train_merge_work.drop(['Id','date','ticker','day','month'],axis=1,inplace=True)

In [917]:
train_merge_work.isnull().sum()

SF1                   0
SF2                   0
SF3                   0
SF4                   0
SF5                   0
SF6                   0
SF7                   0
alpha                 0
sentiment_score    2167
dtype: int64

In [922]:
x = train_merge_work.drop('alpha',axis=1)
y = train_merge_work.alpha
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30,random_state=123)

In [924]:

from datetime import datetime
from sklearn import metrics
from sklearn.metrics import classification_report     
from sklearn.metrics import confusion_matrix     
from sklearn.preprocessing import StandardScaler  # Helps us in Standardizing the data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,recall_score,precision_score

In [925]:
# Lets stick to default Hyperparametrs as of now. We shall tune the best model
log_reg = LogisticRegression(random_state= 42)
svc = SVC(random_state= 42,gamma='auto',probability=True)
rf = RandomForestClassifier(random_state= 42,n_estimators = 100)
ada = AdaBoostClassifier(random_state= 42)
xgb = XGBClassifier(random_state= 42)
dt=DecisionTreeClassifier(criterion = "gini",random_state = 42)
bc = BaggingClassifier(base_estimator=dt, random_state=42)
vc = VotingClassifier(estimators=[('LogisticRegresion', log_reg), ('RandomForest',rf), ('SupportVectorMachine', svc),('AdaBoost',ada),('XGBClassifier',xgb),('DecisionTree',dt)],voting='soft')

In [926]:
# Predict function
def training_predict(trainer, X_train, Y_train, X_test, Y_test):
    scores = {} # Initializing a Dictionary to store evaluation metrics    
    trainer.fit(X_train, Y_train)  # Training
    Y_pred = trainer.predict(X_test) # Predicts  
    scores['Accuracy_test'] = accuracy_score(Y_test , Y_pred)    
    scores['Recall_test'] = recall_score(Y_test , Y_pred,average='macro')    
    scores['Precision_test'] = precision_score(Y_test , Y_pred,average='macro')    
    return scores

In [938]:
x_train.fillna(method='ffill',inplace=True)
x_test.fillna(method='ffill',inplace=True)

/Applications/Anaconda_Installation_Folder/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [939]:
# Ignore the error
import time
start = time.time()
predictors = {} # Initializing a dictionary to store training and testing data
for model in [log_reg, svc, rf,dt,ada, xgb,bc,vc]: # these are our objects that we instantiated above. We loop through them
    model_name = model.__class__.__name__
    predictors[model_name]  = {}
    predictors[model_name] =  training_predict(model, x_train, y_train, x_test, y_test)#Scores that is returned from trainer goes to
                                                                             # the respective predictor   print('Loop Ended at {}'.format(datetime.now().strftime("%H:%M:%S")))
print('\n')    
print('Loop took {} mins'.format((time.time() - start) / 60))  

/Applications/Anaconda_Installation_Folder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/Anaconda_Installation_Folder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/Anaconda_Installation_Folder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/Anaconda_Installation_Folder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th



Loop took 3.3808231671651203 mins


In [974]:
pd.DataFrame(predictors).T

,Accuracy_test,Precision_test,Recall_test
LogisticRegression,0.444705,0.426011,0.303688
SVC,0.648852,0.639461,0.558994
RandomForestClassifier,0.704888,0.664848,0.639092
DecisionTreeClassifier,0.615157,0.568431,0.568808
AdaBoostClassifier,0.528511,0.463064,0.437221
XGBClassifier,0.632313,0.612637,0.545587
BaggingClassifier,0.690200,0.643030,0.628815
VotingClassifier,0.682054,0.644555,0.611065


In [977]:
random_forest=RandomForestClassifier(n_estimators=42)
random_forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=42,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [978]:
y_pred=random_forest.predict(x_test)

In [979]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

In [1]:
print("F1_Score :",f1_score(y_test,y_pred,average='macro'))

NameError: name 'f1_score' is not defined

In [983]:
json_datatwo.head()

,stocktwit_tweet,ticker,timestamp,sentiment_score
0,$CELG nothing to be exited about,$CELG,2018-10-25 14:26:16+00:00,2
1,$AMD yall exhaust your buyer on first green ca...,$AMD,2018-07-13 13:50:39+00:00,2
2,$AMD day traders day.,$AMD,2018-09-25 19:10:54+00:00,2
3,$CBS https://tenor.com/wLB8.gif,$CBS,2018-07-27 22:45:48+00:00,2
4,$MU weak price action so far today. Don’t be a...,$MU,2018-07-31 14:59:06+00:00,4


In [984]:
json_datatwo['date'] = json_datatwo['timestamp'].str.split(' ',expand=True)[0]

In [986]:
json_datatwo['day'] = json_datatwo['date'].str.split('-',expand=True)[2]
json_datatwo['month'] = json_datatwo['date'].str.split('-',expand=True)[1]

In [987]:
json_datatwo=json_datatwo.groupby(by=['day','month','ticker'])['sentiment_score'].agg(lambda x:x.value_counts().index[0]).to_frame()

In [992]:
traintwo['day']=traintwo['date'].str.split('/',expand=True)[0]
traintwo['month']=traintwo['date'].str.split('/',expand=True)[1]

In [993]:
test_merge_work = pd.merge(traintwo, json_datatwo,how='left',left_on = ['ticker','day','month'],right_on=['ticker','day','month'])

In [994]:
test_merge_work.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7,day,month,sentiment_score
0,270007,21/07/18,$INTC,-3.062194,1.223466,1.741714,2.279266,-1.323573,-0.274912,-4.504449,21,07,2
1,270008,05/10/18,$CTSH,0.816263,-2.184408,0.157975,-0.264743,-0.836282,0.046276,0.826353,05,10,2
2,270009,01/10/18,$CB,0.401281,0.091604,0.083411,-1.147041,-0.485223,-0.601060,1.012811,01,10,2
3,270010,24/10/18,$CTAS,-0.783521,1.192929,0.813831,-0.368166,-1.113656,-0.553581,-0.683803,24,10,2
4,270011,27/07/18,$intc,0.796507,0.455341,0.679032,0.354336,-1.799055,0.126153,0.297111,27,07,2


In [995]:
x_train.head()

,SF1,SF2,SF3,SF4,SF5,SF6,SF7,sentiment_score
21056,0.406648,0.565401,-0.344187,-0.117595,0.406803,-0.225448,0.518212,2.0
17272,-0.441948,0.136212,0.108966,-0.210024,0.052615,-1.748684,-0.289019,2.0
18342,0.603605,0.289520,-0.052951,-1.458077,-0.366300,0.351303,1.412169,2.0
24702,-1.891393,-0.927377,0.978116,1.356865,-0.630696,0.740813,-2.725169,2.0
20574,0.922967,0.649260,0.467327,0.789087,-1.439798,-1.318188,0.204886,2.0


In [996]:
test_merge_work = test_merge_work.drop(columns=['Id','date','ticker','day','month'])

In [997]:
y_testpred_rfc=random_forest.predict(test_merge_work)

In [998]:
y_testpred_rfc.shape

(11575,)

In [999]:
df = pd.read_csv("test_factors.csv",sep=',', header=0, na_values='unknown')

In [1000]:
df.head()

,Id,date,ticker,SF1,SF2,SF3,SF4,SF5,SF6,SF7
0,270007,21/07/18,$INTC,-3.062194,1.223466,1.741714,2.279266,-1.323573,-0.274912,-4.504449
1,270008,05/10/18,$CTSH,0.816263,-2.184408,0.157975,-0.264743,-0.836282,0.046276,0.826353
2,270009,01/10/18,$CB,0.401281,0.091604,0.083411,-1.147041,-0.485223,-0.601060,1.012811
3,270010,24/10/18,$CTAS,-0.783521,1.192929,0.813831,-0.368166,-1.113656,-0.553581,-0.683803
4,270011,27/07/18,$intc,0.796507,0.455341,0.679032,0.354336,-1.799055,0.126153,0.297111


In [1001]:
final= pd.DataFrame(list(zip(df['Id'],y_testpred_rfc)),columns=['Id','alpha'])
final.to_csv("harsha_3090.csv",index=False)
final.shape

(11575, 2)